Check the Domain Expansion hand sign: https://www.wikihow.com/Domain-Expansion-Hand-Sign

In [1]:
#pip install opencv-python

In [2]:
#pip install mediapipe

In [3]:
#pip install tensorflow

In [4]:
#pip install pygame

In [8]:
import cv2
import mediapipe as mp
import numpy as np
import pickle
import os
import pygame
import time
import threading
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import tkinter as tk
from tkinter import filedialog, messagebox
from pathlib import Path

class JujutsuKaisenGestureRecognition:
    def __init__(self):
        # Initialize MediaPipe Hands
        self.mp_hands = mp.solutions.hands
        self.hands = self.mp_hands.Hands(
            static_image_mode=False,
            max_num_hands=2,
            min_detection_confidence=0.7,
            min_tracking_confidence=0.7
        )
        self.mp_drawing = mp.solutions.drawing_utils
        
        # Domain definitions 
        self.domain_signs = {
            "infinite_void": "Infinite Void (Gojo Satoru)",
            "chimera_shadow": "Chimera Shadow (Megumi Fushiguro)",
            "malevolent_shrine": "Malevolent Shrine (Sukuna)"
        }
        
        # Paths for data storage
        self.data_dir = "jujutsu_data"
        self.model_path = "jujutsu_model.pkl"
        self.sound_dir = "sounds"
        self.image_dir = "images"
        self.video_dir = "gesture_videos"
        
        # Ensure directories exist
        for directory in [self.data_dir, self.sound_dir, self.image_dir, self.video_dir]:
            if not os.path.exists(directory):
                os.makedirs(directory)
        
        # Initialize pygame for sound
        try:
            pygame.mixer.init()
            self.sounds_available = True
        except:
            print("Sound functionality not available. Continuing without sound effects.")
            self.sounds_available = False
        
        # Load or create model
        self.model = None
        if os.path.exists(self.model_path):
            self.load_model()
        
        # Variables for video recording
        self.recording = False
        self.video_writer = None
        self.current_video_path = ""
        
        # Enhanced detection variables
        self.last_prediction = None
        self.prediction_confidence = 0
        self.prediction_time = 0
        self.detection_cooldown = 2.0
        self.detection_window = 3.0
        self.min_detection_time = 1.5
        
        # For improved prediction smoothing and validation
        self.gesture_buffer = []
        self.gesture_buffer_size = 10
        self.confidence_threshold = 0.8
        self.consistency_threshold = 0.7
        
        # Domain preview images
        self.domain_images = {}
        self.load_domain_images()
        
        # Detection state tracking
        self.gesture_start_time = 0
        self.stable_gesture = None
        self.gesture_confirmed = False
        
        print("✅ Jujutsu Kaisen Domain Expansion recognition system initialized")

    def load_domain_images(self):
        """Load domain expansion character images from the images directory"""
        # Create character images for each domain
        character_data = {
            "infinite_void": {
                "name": "Gojo Satoru",
                "color": (255, 200, 150),  # Light blue/white
                "bg_color": (200, 100, 50)
            },
            "chimera_shadow": {
                "name": "Megumi Fushiguro", 
                "color": (150, 100, 200),  # Purple
                "bg_color": (100, 50, 100)
            },
            "malevolent_shrine": {
                "name": "Sukuna",
                "color": (100, 100, 255),  # Red
                "bg_color": (50, 50, 200)
            }
        }
        
        for domain, data in character_data.items():
            img_path = os.path.join(self.image_dir, f"{domain}.jpg")
            if os.path.exists(img_path):
                try:
                    img = cv2.imread(img_path)
                    if img is not None:
                        # Resize to 200x200 for preview
                        img = cv2.resize(img, (200, 200))
                        self.domain_images[domain] = img
                        print(f"✅ Loaded domain image: {domain}.jpg")
                    else:
                        self.create_character_image(domain, data)
                except Exception as e:
                    print(f"❌ Error loading {img_path}: {e}")
                    self.create_character_image(domain, data)
            else:
                self.create_character_image(domain, data)

    def create_character_image(self, domain, data):
        """Create a stylized character image for domain preview"""
        # Create 200x200 image
        img = np.ones((200, 200, 3), dtype=np.uint8)
        
        # Create gradient background
        for y in range(200):
            for x in range(200):
                # Radial gradient from center
                center_x, center_y = 100, 100
                distance = np.sqrt((x - center_x)**2 + (y - center_y)**2)
                fade = max(0, 1 - distance / 140)
                
                color = np.array(data["bg_color"]) * fade + np.array([20, 20, 20]) * (1 - fade)
                img[y, x] = color.astype(np.uint8)
        
        # Add character silhouette/face outline
        if domain == "infinite_void":
            # Gojo's distinctive features - blindfold/eyes
            cv2.ellipse(img, (100, 80), (40, 25), 0, 0, 360, (255, 255, 255), -1)
            cv2.ellipse(img, (85, 75), (8, 6), 0, 0, 360, (0, 200, 255), -1)
            cv2.ellipse(img, (115, 75), (8, 6), 0, 0, 360, (0, 200, 255), -1)
            
        elif domain == "chimera_shadow":
            # Megumi's spiky hair silhouette
            points = np.array([[100, 40], [85, 60], [70, 50], [90, 80], [100, 70], 
                              [110, 80], [130, 50], [115, 60], [100, 40]], np.int32)
            cv2.fillPoly(img, [points], (80, 80, 120))
            
        elif domain == "malevolent_shrine":
            # Sukuna's markings
            cv2.line(img, (80, 60), (120, 60), (255, 100, 100), 3)
            cv2.line(img, (80, 80), (120, 80), (255, 100, 100), 3)
            cv2.line(img, (85, 100), (115, 100), (255, 100, 100), 3)
        
        # Add domain name
        cv2.putText(img, data["name"], (30, 180), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
        
        self.domain_images[domain] = img
        print(f"✅ Created character image for {domain}")

    def extract_hand_features(self, frame):
        """Extract features from hand landmarks using MediaPipe Hands"""
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = self.hands.process(image_rgb)
        
        features = np.zeros(84)
        hands_detected = False
        hand_count = 0
        
        if results.multi_hand_landmarks:
            hands_detected = True
            hand_count = len(results.multi_hand_landmarks)
            
            # Draw hand landmarks
            for hand_landmarks in results.multi_hand_landmarks:
                self.mp_drawing.draw_landmarks(
                    frame,
                    hand_landmarks,
                    self.mp_hands.HAND_CONNECTIONS,
                    self.mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2),
                    self.mp_drawing.DrawingSpec(color=(255, 255, 255), thickness=2)
                )
            
            # Extract features from up to 2 hands
            for hand_idx, hand_landmarks in enumerate(results.multi_hand_landmarks[:2]):
                for i, landmark in enumerate(hand_landmarks.landmark):
                    base_idx = hand_idx * 42
                    features[base_idx + i*2] = landmark.x
                    features[base_idx + i*2 + 1] = landmark.y
        
        return features, frame, hands_detected, hand_count

    def update_gesture_buffer(self, prediction, confidence):
        """Update gesture buffer and check for consistent detection"""
        current_time = time.time()
        
        self.gesture_buffer.append({
            'prediction': prediction,
            'confidence': confidence,
            'timestamp': current_time
        })
        
        if len(self.gesture_buffer) > self.gesture_buffer_size:
            self.gesture_buffer.pop(0)
        
        self.gesture_buffer = [
            entry for entry in self.gesture_buffer 
            if current_time - entry['timestamp'] < self.detection_window
        ]
        
        if len(self.gesture_buffer) >= 5:
            domain_counts = {}
            total_confidence = {}
            
            for entry in self.gesture_buffer:
                domain = entry['prediction']
                if domain not in domain_counts:
                    domain_counts[domain] = 0
                    total_confidence[domain] = 0
                domain_counts[domain] += 1
                total_confidence[domain] += entry['confidence']
            
            max_count = max(domain_counts.values())
            total_predictions = len(self.gesture_buffer)
            consistency_ratio = max_count / total_predictions
            
            if consistency_ratio >= self.consistency_threshold:
                dominant_domain = max(domain_counts.keys(), key=lambda k: domain_counts[k])
                avg_confidence = total_confidence[dominant_domain] / domain_counts[dominant_domain]
                
                if dominant_domain != self.stable_gesture:
                    self.stable_gesture = dominant_domain
                    self.gesture_start_time = current_time
                    self.gesture_confirmed = False
                
                elif current_time - self.gesture_start_time >= self.min_detection_time:
                    if not self.gesture_confirmed and avg_confidence >= self.confidence_threshold:
                        self.gesture_confirmed = True
                        return dominant_domain, avg_confidence
        
        return None, 0

    def show_domain_preview(self, frame, domain, confidence):
        """Show character preview in upper right corner like in the image"""
        if domain in self.domain_images:
            h, w = frame.shape[:2]
            preview_size = 150
            
            # Position in top-right corner with margin
            start_x = w - preview_size - 20
            start_y = 30
            
            # Get and resize domain image
            domain_img = self.domain_images[domain].copy()
            preview_img = cv2.resize(domain_img, (preview_size, preview_size))
            
            # Add green border (like in your image)
            border_thickness = 3
            cv2.rectangle(frame, 
                         (start_x - border_thickness, start_y - border_thickness),
                         (start_x + preview_size + border_thickness, start_y + preview_size + border_thickness),
                         (0, 255, 0), border_thickness)
            
            # Place the character image
            frame[start_y:start_y+preview_size, start_x:start_x+preview_size] = preview_img
            
            # Add confidence text below image (green text like in your image)
            conf_text = f"Confidence: {confidence:.2f}"
            cv2.putText(frame, conf_text, 
                       (start_x, start_y + preview_size + 20),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    def record_gesture_video(self):
        """Record gesture videos for training"""
        print("\n== VIDEO RECORDING MODE ==")
        
        cap = cv2.VideoCapture(0)
        if not cap.isOpened():
            print("❌ Error: Could not open camera.")
            return
            
        cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
        cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
        fps = int(cap.get(cv2.CAP_PROP_FPS)) or 30
        
        print(f"✅ Camera initialized (FPS: {fps})")
        print("Controls: 'R' = Record, 'Q' = Quit")
        
        cv2.namedWindow("Gesture Video Recorder", cv2.WINDOW_NORMAL)
        video_count = 1
        
        while True:
            ret, frame = cap.read()
            if not ret:
                break
                
            frame = cv2.flip(frame, 1)
            
            try:
                features, processed_frame, hands_detected, hand_count = self.extract_hand_features(frame)
            except:
                processed_frame = frame.copy()
                hands_detected = False
                hand_count = 0
            
            # Display recording status
            if self.recording:
                cv2.putText(processed_frame, f"🔴 RECORDING - Video {video_count}", 
                           (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
                if hasattr(self, 'recording_start_time'):
                    elapsed = time.time() - self.recording_start_time
                    cv2.putText(processed_frame, f"Time: {elapsed:.1f}s", 
                               (450, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
                
                if self.video_writer:
                    self.video_writer.write(processed_frame)
            else:
                cv2.putText(processed_frame, "Press 'R' to start recording", 
                           (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            
            hand_status = f"👋 Hands: {hand_count}" if hands_detected else "❌ No Hands"
            color = (0, 255, 0) if hands_detected else (0, 0, 255)
            cv2.putText(processed_frame, hand_status, (10, 70), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
            
            cv2.imshow("Gesture Video Recorder", processed_frame)
            
            key = cv2.waitKey(1) & 0xFF
            if key == ord('q'):
                break
            elif key == ord('r'):
                if self.recording:
                    self.recording = False
                    if self.video_writer:
                        self.video_writer.release()
                        self.video_writer = None
                    print(f"✅ Video saved: {self.current_video_path}")
                    video_count += 1
                else:
                    timestamp = time.strftime("%Y%m%d_%H%M%S")
                    video_filename = f"gesture_video_{video_count}_{timestamp}.mp4"
                    self.current_video_path = os.path.join(self.video_dir, video_filename)
                    
                    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
                    self.video_writer = cv2.VideoWriter(
                        self.current_video_path, fourcc, fps, 
                        (processed_frame.shape[1], processed_frame.shape[0])
                    )
                    
                    if self.video_writer.isOpened():
                        self.recording = True
                        self.recording_start_time = time.time()
                        print(f"🔴 Started recording: {video_filename}")
        
        if self.recording and self.video_writer:
            self.video_writer.release()
        
        cap.release()
        cv2.destroyAllWindows()

    def extract_features_from_video(self, video_path, label):
        """Extract hand features from video file"""
        print(f"Processing video: {os.path.basename(video_path)}")
        
        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            print(f"❌ Error: Could not open video {video_path}")
            return [], []
        
        features_list = []
        labels_list = []
        frame_count = 0
        valid_frames = 0
        
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            
            frame_count += 1
            if frame_count % 2 != 0:  # Process every other frame
                continue
            
            try:
                frame = cv2.flip(frame, 1)
                features, _, hands_detected, hand_count = self.extract_hand_features(frame)
                
                if hands_detected and hand_count > 0:
                    features_list.append(features)
                    labels_list.append(label)
                    valid_frames += 1
            except Exception as e:
                continue
        
        cap.release()
        print(f"✅ Extracted {valid_frames} valid frames")
        return features_list, labels_list

    def train_from_videos(self):
        """Train model from recorded videos"""
        print("\n== TRAIN FROM VIDEOS ==")
        
        # Get all video files
        video_files = []
        for file in os.listdir(self.video_dir):
            if file.endswith(('.mp4', '.avi', '.mov')):
                video_files.append(os.path.join(self.video_dir, file))
        
        if not video_files:
            print("No video files found in gesture_videos directory.")
            return
        
        # Label videos
        labeled_videos = []
        domains = list(self.domain_signs.keys())
        
        print(f"Found {len(video_files)} videos")
        print("Available domains:")
        for i, domain in enumerate(domains):
            print(f"{i+1}: {self.domain_signs[domain]}")
        
        for video_path in video_files:
            video_name = os.path.basename(video_path)
            print(f"\nVideo: {video_name}")
            
            while True:
                try:
                    label_choice = input(f"Enter domain number (1-{len(domains)}) or 'skip': ").strip()
                    
                    if label_choice.lower() == 'skip':
                        break
                    
                    domain_idx = int(label_choice) - 1
                    if 0 <= domain_idx < len(domains):
                        domain = domains[domain_idx]
                        labeled_videos.append((video_path, domain))
                        print(f"✅ {video_name} -> {self.domain_signs[domain]}")
                        break
                    else:
                        print("Invalid choice.")
                except ValueError:
                    print("Invalid input.")
        
        if not labeled_videos:
            print("No labeled videos available.")
            return
        
        # Extract features
        all_features = []
        all_labels = []
        
        for video_path, label in labeled_videos:
            features, labels = self.extract_features_from_video(video_path, label)
            all_features.extend(features)
            all_labels.extend(labels)
        
        if not all_features:
            print("No features extracted.")
            return
        
        X = np.array(all_features)
        y = np.array(all_labels)
        
        print(f"Total training samples: {len(X)}")
        
        # Train model
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        
        self.model = RandomForestClassifier(n_estimators=100, random_state=42)
        self.model.fit(X_train, y_train)
        
        # Evaluate
        y_pred = self.model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        print(f"Model accuracy: {accuracy:.2f}")
        
        # Save model
        with open(self.model_path, 'wb') as f:
            pickle.dump(self.model, f)
        print(f"✅ Model saved to {self.model_path}")

    def load_model(self):
        """Load trained model"""
        try:
            with open(self.model_path, 'rb') as f:
                self.model = pickle.load(f)
            print(f"✅ Loaded model from {self.model_path}")
        except Exception as e:
            print(f"❌ Could not load model: {e}")
            self.model = None

    def start_detection(self):
        """Start real-time domain expansion detection"""
        if self.model is None:
            print("❌ No model loaded. Please train a model first.")
            return
        
        print("\n== DOMAIN EXPANSION DETECTION ==")
        print("🔥 Starting real-time detection")
        
        cap = cv2.VideoCapture(0)
        if not cap.isOpened():
            print("❌ Error: Could not open camera.")
            return
            
        cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
        cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
        
        cv2.namedWindow("Jujutsu Kaisen Domain Detection", cv2.WINDOW_NORMAL)
        
        # Reset detection state
        self.gesture_buffer = []
        self.stable_gesture = None
        self.gesture_confirmed = False
        
        while True:
            ret, frame = cap.read()
            if not ret:
                break
                
            frame = cv2.flip(frame, 1)
            current_time = time.time()
            
            try:
                # Normal detection only
                features, frame, hands_detected, hand_count = self.extract_hand_features(frame)
                
                if hands_detected and hand_count > 0:
                    if current_time - self.prediction_time > self.detection_cooldown:
                        prediction = self.model.predict([features])[0]
                        probs = self.model.predict_proba([features])[0]
                        confidence = np.max(probs)
                        
                        confirmed_domain, confirmed_confidence = self.update_gesture_buffer(prediction, confidence)
                        
                        if confirmed_domain:
                            print(f"🔥 DOMAIN DETECTED: {self.domain_signs[confirmed_domain]} (confidence: {confirmed_confidence:.2f})")
                            self.last_prediction = confirmed_domain
                            self.prediction_confidence = confirmed_confidence
                            self.prediction_time = current_time
                            
                            # Reset detection state
                            self.gesture_buffer = []
                            self.stable_gesture = None
                            self.gesture_confirmed = False
                    
                    # Show preview when detecting
                    if self.stable_gesture and not self.gesture_confirmed:
                        time_held = current_time - self.gesture_start_time
                        progress = min(1.0, time_held / self.min_detection_time)
                        
                        # Show character preview in upper right
                        avg_conf = sum(e['confidence'] for e in self.gesture_buffer if e['prediction'] == self.stable_gesture) / max(1, sum(1 for e in self.gesture_buffer if e['prediction'] == self.stable_gesture))
                        self.show_domain_preview(frame, self.stable_gesture, avg_conf)
                        
                        # Progress bar
                        bar_width = 200
                        bar_x = (frame.shape[1] - bar_width) // 2
                        bar_y = 50
                        
                        cv2.rectangle(frame, (bar_x, bar_y), (bar_x + bar_width, bar_y + 10), (100, 100, 100), -1)
                        cv2.rectangle(frame, (bar_x, bar_y), (bar_x + int(bar_width * progress), bar_y + 10), (0, 255, 0), -1)
                        cv2.putText(frame, f"gesture: {progress*100:.0f}%", 
                                   (bar_x, bar_y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
                
                # Status display
                if hands_detected:
                    cv2.putText(frame, f"👋 Hands Detected ({hand_count}) - Ready for detection", 
                               (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
                    
                    # Show last detection
                    if self.last_prediction and current_time - self.prediction_time < 10:
                        detection_text = f"Last: {self.domain_signs[self.last_prediction]} ({self.prediction_confidence:.2f})"
                        cv2.putText(frame, detection_text, (10, 60), 
                                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 1)
                else:
                    cv2.putText(frame, "❌ Show hands for detection", 
                               (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
                    self.gesture_buffer = []
                    self.stable_gesture = None
                    self.gesture_confirmed = False
                
                # Instructions
                cv2.putText(frame, "Hold gesture steady for detection | Press 'Q' to quit", 
                           (10, frame.shape[0] - 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
                
                cv2.imshow("Jujutsu Kaisen Domain Detection", frame)
                
            except Exception as e:
                print(f"Detection error: {e}")
                cv2.putText(frame, f"Error: {str(e)[:30]}...", 
                           (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)
                cv2.imshow("Jujutsu Kaisen Domain Detection", frame)
            
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        
        cap.release()
        cv2.destroyAllWindows()

    def run(self):
        """Main menu"""
        while True:
            print("\n" + "="*50)
            print("🔥 JUJUTSU KAISEN DOMAIN DETECTION 🔥")
            print("="*50)
            print("1: 📹 Record Gesture Videos")
            print("2: 📊 Train Model from Videos") 
            print("3: 🎯 Start Domain Detection")
            print("4: 🚪 Exit")
            print("="*50)
            
            try:
                choice = input("Select option (1-4): ").strip()
                
                if choice == '1':
                    self.record_gesture_video()
                elif choice == '2':
                    self.train_from_videos()
                elif choice == '3':
                    self.start_detection()
                elif choice == '4':
                    print("👋 Exiting...")
                    break
                else:
                    print("❌ Invalid choice.")
                    
            except KeyboardInterrupt:
                print("\n👋 Exiting...")
                break
            except Exception as e:
                print(f"❌ Error: {e}")

if __name__ == "__main__":
    try:
        print("🔥 Starting Jujutsu Kaisen Domain Detection System...")
        jjk = JujutsuKaisenGestureRecognition()
        jjk.run()
    except Exception as e:
        print(f"❌ Fatal error: {e}")
        import traceback
        traceback.print_exc()

🔥 Starting Jujutsu Kaisen Domain Detection System...
✅ Loaded model from jujutsu_model.pkl
✅ Loaded domain image: infinite_void.jpg
✅ Loaded domain image: chimera_shadow.jpg
✅ Loaded domain image: malevolent_shrine.jpg
✅ Jujutsu Kaisen Domain Expansion recognition system initialized

🔥 JUJUTSU KAISEN DOMAIN DETECTION 🔥
1: 📹 Record Gesture Videos
2: 📊 Train Model from Videos
3: 🎯 Start Domain Detection
4: 🚪 Exit


Select option (1-4):  3



== DOMAIN EXPANSION DETECTION ==
🔥 Starting real-time detection
🔥 DOMAIN DETECTED: Malevolent Shrine (Sukuna) (confidence: 0.81)
🔥 DOMAIN DETECTED: Malevolent Shrine (Sukuna) (confidence: 0.87)
🔥 DOMAIN DETECTED: Infinite Void (Gojo Satoru) (confidence: 0.81)
🔥 DOMAIN DETECTED: Infinite Void (Gojo Satoru) (confidence: 1.00)

🔥 JUJUTSU KAISEN DOMAIN DETECTION 🔥
1: 📹 Record Gesture Videos
2: 📊 Train Model from Videos
3: 🎯 Start Domain Detection
4: 🚪 Exit


Select option (1-4):  4


👋 Exiting...
